<a href="https://colab.research.google.com/github/codeprogredire/tensorflow/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download the pre-trained weights. No top means it excludes the fully connected layer it uses for classification.
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2024-04-17 05:55:20--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.115.207, 172.253.122.207, 172.253.63.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.115.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   107MB/s    in 0.8s    

2024-04-17 05:55:21 (107 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

# Set the weights file you downloaded into a variable
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

#Initialize the base model
#Set the input shape and remove the dense layers
pre_trained_model=InceptionV3(input_shape=(150,150,3),
                              include_top=False,
                              weights=None)

#load the pretrained weights you downloaded
pre_trained_model.load_weights(local_weights_file)

#Freeze the weights of the layers
for layer in pre_trained_model.layers:
  layer.trainable=False

In [ ]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 74, 74, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 74, 74, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 74, 74, 32)           0         ['batch_normalizati

In [ ]:
#Choose 'mixed7' as the last layer of your base model
last_layer=pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output=last_layer.output

last layer output shape:  (None, 7, 7, 768)


# Add dense layers to your classifier

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

#Flatten the output layer to one dimension
x=layers.Flatten()(last_output)

#Add a fully connected layer with 1024 hidden units and relu activation
x=layers.Dense(1024,activation='relu')(x)

# Add a dropout rate of 0.2
x=layers.Dropout(0.2)(x)

# Add a final sigmoid layer for classification
x=layers.Dense(1,activation='sigmoid')(x)

#Append the dense network to the base model
model=Model(pre_trained_model.input,x)

#print the model summary
model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 74, 74, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 74, 74, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 74, 74, 32)           0         ['batch_normalization[0][0

In [ ]:
# set the training parameter
model.compile(optimizer=RMSprop(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Prepare the dataset

In [ ]:
# Download the dataset
!wget https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip

--2024-04-17 06:04:44--  https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.111.207, 142.251.16.207, 172.253.62.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.111.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M   146MB/s    in 0.4s    

2024-04-17 06:04:44 (146 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Extract the archive
zip_ref = zipfile.ZipFile("./cats_and_dogs_filtered.zip", 'r')
zip_ref.extractall("tmp/")
zip_ref.close()

# Define our example directories and files
base_dir = 'tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')

# Directory with training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (150, 150))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary',
                                                          target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
# Train the model.
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 10,
            validation_steps = 50,
            verbose = 1)

Epoch 1/10
100/100 [==============================] - 179s 2s/step - loss: 0.2093 - accuracy: 0.9100 - val_loss: 0.0940 - val_accuracy: 0.9660
Epoch 2/10
100/100 [==============================] - 180s 2s/step - loss: 0.1787 - accuracy: 0.9270 - val_loss: 0.1022 - val_accuracy: 0.9640
Epoch 3/10
100/100 [==============================] - 179s 2s/step - loss: 0.1797 - accuracy: 0.9290 - val_loss: 0.0915 - val_accuracy: 0.9670
Epoch 4/10
100/100 [==============================] - 177s 2s/step - loss: 0.1593 - accuracy: 0.9395 - val_loss: 0.0873 - val_accuracy: 0.9690
Epoch 5/10
100/100 [==============================] - 176s 2s/step - loss: 0.1743 - accuracy: 0.9295 - val_loss: 0.0899 - val_accuracy: 0.9650
Epoch 6/10
100/100 [==============================] - 179s 2s/step - loss: 0.1640 - accuracy: 0.9400 - val_loss: 0.0775 - val_accuracy: 0.9700
Epoch 7/10
100/100 [==============================] - 174s 2s/step - loss: 0.1201 - accuracy: 0.9550 - val_loss: 0.0891 - val_accuracy: 0.9690